# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [11]:
import numpy as np
import pandas as pd
from scipy import stats

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [6]:
pokemon['Legendary'].value_counts()
# 65 legendary pokemons vs 735 non-legendary pokemons

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [8]:
# mean of total points:
pokemon.groupby('Legendary').mean()['Total']
# Legendary: 637.384615
# Non-legendary: 417.213605

Legendary
False    417.213605
True     637.384615
Name: Total, dtype: float64

In [9]:
# standard deviation of total points:
pokemon.groupby('Legendary').std()['Total']
# Legendary: 60.937389
# Non-legendary: 106.760417

Legendary
False    106.760417
True      60.937389
Name: Total, dtype: float64

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [18]:
legendary = pokemon[pokemon['Legendary'] == True]['Total']
non_legendary = pokemon[pokemon['Legendary'] == False]['Total']

In [19]:
stats.ttest_ind(legendary, non_legendary, nan_policy='omit', equal_var = False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957444e-47)

What do you conclude from this test? Write your conclusions below.

In [6]:
# Since the p-value is lower than 0.05, we reject the null hypothesis and we accept the alternative hypothesis that
# there is a significant difference between the means of the total poins of legendary and non-legendary pokemons

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [20]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [23]:
water = pokemon[pokemon['Type 1'] == 'Water']['Total']
non_water = pokemon[pokemon['Legendary'] != 'Water']['Total']

In [32]:
print('The total points mean in water pokemon is', round(water.mean(), 2), 'and the standard deviation is', round(water.std(), 2))
print('The total points mean in non-water pokemon is', round(non_water.mean(), 2), 'and the standard deviation is', round(non_water.std(), 2))

The total points mean in water pokemon is 430.46 and the standard deviation is 113.19
The total points mean in non-water pokemon is 435.1 and the standard deviation is 119.96


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [33]:
stats.ttest_ind(water, non_water, nan_policy='omit', equal_var = True)

Ttest_indResult(statistic=-0.3865645058398148, pvalue=0.699169009876486)

Write your conclusion below.

In [34]:
# The p-value is much higher than 0.5, so we can't reject the null hypothesis. 
# We assume that there is no significant difference between the means of total points 
# of water pokemons and non-water pokemons

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [35]:
stats.ttest_rel(pokemon.Attack, pokemon.Defense)

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [12]:
# Because the p-value is lower than 0.05, we reject the null hypothesis and accept the alternative hypothesis that
# there is a significant difference between the means of Attack and Defense. 

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [37]:
stats.ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [14]:
# Because the p-value is higher than 0.05, we accept the null hypothesis that
# there is no significant difference between the means of Special Attack and Special Defense. 

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [39]:
stats.ttest_1samp(pokemon.Attack - pokemon.Defense, 0)

Ttest_1sampResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

In [41]:
stats.ttest_rel(pokemon.Attack, pokemon.Defense) == stats.ttest_1samp(pokemon.Attack - pokemon.Defense, 0)
# the results of the two tests are exactly the same

True

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

